In [2]:
import pandas as pd
import pybamm
import pybop

In [9]:
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        # "Current function [A]": 5,
        # "Initial SoC": 0.5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 2},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

file_loc = r"../hppc_lut/G1/battery_G1_cycle_2_pulse_0_lut.csv"
df = pd.read_csv(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(),
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)

r_guess = 0.001  
c_guess = 5000  

parameters = pybop.Parameters(
    pybop.Parameter("R0 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess/2), bounds=[1e-5, 0.1]),
    pybop.Parameter("R1 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess/2), bounds=[1e-5, 0.1]),
    pybop.Parameter("R2 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess/2), bounds=[1e-5, 0.1]),
    pybop.Parameter("C1 [F]", prior=pybop.Gaussian(c_guess, c_guess/5), bounds=[100, 50000]),
    pybop.Parameter("C2 [F]", prior=pybop.Gaussian(c_guess/2, c_guess/10), bounds=[100, 50000]),
)

initial_voltage = df["Voltage"].to_numpy()[0]
print(f"Initial Voltage: {initial_voltage}")

initial_conditions = {
    "Initial open-circuit voltage [V]": initial_voltage,
    "Initial SoC": initial_voltage / 4.2  # Approximate SoC based on voltage
}

model.build(
    initial_state={"Initial SoC": 0.95}
)
problem = pybop.FittingProblem(
    model,
    parameters,
    dataset,
)

cost = pybop.SumSquaredError(problem)

Initial Voltage: 4.186875343322754


In [14]:
optim = pybop.XNES( 
    cost = cost,
    sigma0=[1e-5, 1e-5, 1e-5, 100, 100], 
    max_unchanged_iterations=50,
    max_iterations=100
)
results = optim.run()

Halt: No significant change for 50 iterations.


ValueError: Optimised parameters do not produce a finite cost value

In [15]:
import pybamm
import pybop
import pandas as pd
import numpy as np
from scipy.stats import loguniform

# Parameter set definition with voltage-appropriate values
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        # Conservative initial guesses based on high voltage
        "R0 [Ohm]": 0.005,
        "R1 [Ohm]": 0.002,
        "C1 [F]": 1000,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)

parameter_set.update(
    {
        "R2 [Ohm]": 0.002,
        "C2 [F]": 1000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

# Custom transformer for parameters
class LogTransformer:
    def __init__(self, lower, upper):
        self.lower = lower
        self.upper = upper
        self.log_lower = np.log10(lower)
        self.log_upper = np.log10(upper)
    
    def transform(self, x):
        return 10 ** (x * (self.log_upper - self.log_lower) + self.log_lower)
    
    def inverse_transform(self, x):
        return (np.log10(x) - self.log_lower) / (self.log_upper - self.log_lower)

def setup_fitting_problem(df, model):
    # Create dataset with minimal preprocessing
    dataset = pybop.Dataset(
        {
            "Time [s]": df["Time"].to_numpy(),
            "Current function [A]": df["Current"].to_numpy(),
            "Voltage [V]": df["Voltage"].to_numpy(),
        }
    )
    
    # Set up parameter transformers
    r_transformer = LogTransformer(1e-4, 1e-2)  # Resistance range
    c_transformer = LogTransformer(100, 5000)   # Capacitance range
    
    # Create parameters with transformed bounds
    parameters = pybop.Parameters(
        pybop.Parameter("R0 [Ohm]", 
                       prior=pybop.Uniform(0, 1),  # Will be transformed
                       bounds=[0, 1]),
        pybop.Parameter("R1 [Ohm]", 
                       prior=pybop.Uniform(0, 1),
                       bounds=[0, 1]),
        pybop.Parameter("R2 [Ohm]", 
                       prior=pybop.Uniform(0, 1),
                       bounds=[0, 1]),
        pybop.Parameter("C1 [F]", 
                       prior=pybop.Uniform(0, 1),
                       bounds=[0, 1]),
        pybop.Parameter("C2 [F]", 
                       prior=pybop.Uniform(0, 1),
                       bounds=[0, 1])
    )
    
    # Custom cost function with parameter transformation
    class TransformedCost(pybop.SumSquaredError):
        def __call__(self, x):
            # Transform parameters back to original space
            transformed_x = np.array([
                r_transformer.transform(x[0]),  # R0
                r_transformer.transform(x[1]),  # R1
                r_transformer.transform(x[2]),  # R2
                c_transformer.transform(x[3]),  # C1
                c_transformer.transform(x[4])   # C2
            ])
            return super().__call__(transformed_x)
    
    return dataset, parameters, TransformedCost(pybop.FittingProblem(model, parameters, dataset))

def run_optimization(file_loc):
    # Load data
    df = pd.read_csv(file_loc)
    df = df.drop_duplicates(subset=["Time"], keep="first")
    df["Time"] = df["Time"] - df["Time"].iloc[0]
    
    # Initialize model with careful SoC estimation
    initial_voltage = df["Voltage"].iloc[0]
    print(f"Initial Voltage: {initial_voltage}")
    
    # More careful SoC estimation
    initial_soc = (initial_voltage - 3.0) / (4.2 - 3.0)
    initial_soc = np.clip(initial_soc, 0.05, 0.95)  # Keep away from extremes
    
    model.build(initial_state={"Initial SoC": initial_soc})
    
    # Set up optimization
    dataset, parameters, cost = setup_fitting_problem(df, model)
    
    # Use both CMAES and XNES with careful settings
    optimizers = [
        ("CMAES", pybop.CMAES(
            cost=cost,
            sigma0=0.05,
            population_size=10,
            max_iterations=50
        )),
        ("XNES", pybop.XNES(
            cost=cost,
            sigma0=[0.05] * 5,
            max_unchanged_iterations=20,
            max_iterations=50
        ))
    ]
    
    best_result = None
    best_cost = float('inf')
    
    for name, optim in optimizers:
        try:
            print(f"\nTrying {name} optimizer...")
            result = optim.run()
            if result.final_cost < best_cost:
                best_result = result
                best_cost = result.final_cost
                print(f"New best cost: {best_cost}")
        except Exception as e:
            print(f"{name} optimization failed: {str(e)}")
            continue
    
    return best_result, model, dataset

# Run optimization
file_loc = "../hppc_lut/G1/battery_G1_cycle_2_pulse_0_lut.csv"
results, model, dataset = run_optimization(file_loc)

if results is not None:
    # Transform parameters back to original space
    r_transformer = LogTransformer(1e-4, 1e-2)
    c_transformer = LogTransformer(100, 5000)
    
    transformed_params = [
        r_transformer.transform(results.x[0]),
        r_transformer.transform(results.x[1]),
        r_transformer.transform(results.x[2]),
        c_transformer.transform(results.x[3]),
        c_transformer.transform(results.x[4])
    ]
    
    print("\nOptimized parameters:")
    for param, value in zip(["R0", "R1", "R2", "C1", "C2"], transformed_params):
        print(f"{param}: {value}")

Initial Voltage: 4.186875343322754

Trying CMAES optimizer...

----------------------------------------
Unexpected termination.
Current score: (inf, inf)
Current position:
 4.25548272542419226e-01
 9.25376388875935318e-01
 1.01217256555338531e-01
 1.52281800633863096e-01
 1.51887183581566942e-01
----------------------------------------
CMAES optimization failed: setup_fitting_problem.<locals>.TransformedCost.__call__() got an unexpected keyword argument 'calculate_grad'

Trying XNES optimizer...

----------------------------------------
Unexpected termination.
Current score: (inf, inf)
Current position:
 4.25548272542419226e-01
 9.25376388875935318e-01
 1.01217256555338531e-01
 1.52281800633863096e-01
 1.51887183581566942e-01
----------------------------------------
XNES optimization failed: setup_fitting_problem.<locals>.TransformedCost.__call__() got an unexpected keyword argument 'calculate_grad'
